In [1]:
import jax
import sys
sys.path.append("..")
from jaxrl_m.typing import *
from jaxrl_m.networks import MLP, get_latent, default_init, ensemblize
import equinox as eqx
import equinox.nn as nn
import functools

In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

import jax
import jax.numpy as jnp
import equinox as eqx
import equinox.nn as nn
import optax

import ott
from ott.solvers.linear import sinkhorn
from ott.geometry import pointcloud

import matplotlib.pyplot as plt
import pprint


In [10]:
from src.agents.iql_equinox import GaussianIntentPolicy

key = jax.random.PRNGKey(42)
policy = GaussianIntentPolicy(key=key, state_dim=29, intent_dim=256, hidden_dims=[10, 10])

In [11]:
dist = policy(jax.random.normal(key=key, shape=(29, )))


In [14]:
import jax

sample_key, key = jax.random.split(key, 2)
dist.sample_and_log_prob(seed=sample_key)[0]

Array([ 0.79226196, -0.8326498 , -0.27477828, -1.4187186 ,  1.3861606 ,
        0.55840826,  0.44638506,  0.8171458 ,  0.96836233,  1.1152549 ,
        1.3663728 , -0.48084542,  0.8476486 ,  1.5169784 ,  1.0242839 ,
        0.35271466, -0.363487  , -0.8979553 , -0.9768782 ,  1.6541424 ,
       -0.65059   ,  0.02092186,  2.060648  ,  0.3458615 ,  2.1372218 ,
       -1.6125547 , -0.4665095 ,  0.14492488, -1.3469902 , -0.9702675 ,
        0.24499813, -0.45002604, -1.9654341 ,  0.02293885, -1.9163194 ,
        2.1501043 , -1.2322844 ,  0.33090952,  0.47050998, -0.22664222,
       -0.42976975,  0.9717772 ,  0.29004234,  1.7527819 ,  0.69897115,
       -0.22178045, -2.2419178 ,  0.6496175 , -0.05749149,  0.568928  ,
        2.0610597 , -0.04300588,  0.39271545, -0.02689009, -0.8507596 ,
        0.972741  , -1.153822  ,  2.06174   , -0.9770966 , -0.5826562 ,
       -0.30460912, -1.170865  ,  1.7069314 , -1.4290427 , -0.5017443 ,
       -0.5121806 ,  0.53079665, -1.4503367 , -0.6276568 ,  1.30

In [4]:
loaded_phi_net = eqx.tree_deserialise_leaves("/home/m_bobrin/GOTIL/icvf_phi_300k_umaze.eqx", phi_net)

In [6]:
@eqx.filter_vmap
def ensemblize(keys):
    return MultilinearVF_EQX(key=keys, state_dim=29, hidden_dims=[256, 256], pretrained_phi=loaded_phi_net)

ensemble_net = ensemblize(jax.random.split(key1, 2))